In [55]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, mean_squared_error

In [56]:
data = pd.read_excel('Summary_of_Results_5_6_2023-2.xlsx')
data.head()

,CERT,CHCLASS1,CITYST,COST,FAILDATE,FIN,ID,NAME,QBFASSET,QBFDEP,RESTYPE,RESTYPE1,SAVR
0,14228,NM,"CARLISLE, IA",11574.0,1/14/2000,4641,3524,HARTFORD-CARLISLE SB,105044,68549,FAILURE,PI,BIF
1,27886,SB,"ATLANTA, GA",1322.0,3/10/2000,6003,3525,MUTUAL FEDERAL SAVINGS BANK OF ATLANTA,31479,32234,FAILURE,PA,SAIF
2,25056,N,"RIDGECREST, CA",617.0,6/2/2000,4642,3526,MONUMENT NATIONAL BANK,7923,7680,FAILURE,PA,BIF
3,10985,NM,"ALMELUND, MN",1363.0,7/14/2000,4643,3527,TOWN & COUNTRY BANK OF ALMELUND,25942,26667,FAILURE,PI,BIF
4,338,NM,"FALKNER, MS",14592.0,9/29/2000,4644,3528,BANK OF FALKNER,85485,77140,FAILURE,PI,BIF


In [57]:
data.columns

Index(['CERT', 'CHCLASS1', 'CITYST', 'COST', 'FAILDATE', 'FIN', 'ID', 'NAME',
       'QBFASSET', 'QBFDEP', 'RESTYPE', 'RESTYPE1', 'SAVR'],
      dtype='object')

In [58]:
data2 = pd.read_csv('bank-data.csv')
data2.head()

,CERT,CHCLASS1,CITYST,COST,FAILDATE,FIN,ID,NAME,QBFASSET,QBFDEP,RESTYPE,RESTYPE1,SAVR
0,NaN,NM,"EAST PEORIA, IL",NaN,5/28/1934,0,1,FON DU LAC STATE BANK,374.0,238.0,FAILURE,PO,FDIC
1,NaN,NM,"GRANTWOOD, NJ",NaN,1/3/1935,0,10,CLIFFSIDE PARK TITLE GUARANTEE & TRUST CO.,2305.0,590.0,FAILURE,PO,FDIC
2,NaN,NM,"CUMMINGS, ND",NaN,12/21/1936,0,100,THE CUMMINGS STATE BANK,NaN,30.0,FAILURE,P&A,FDIC
3,13797.0,NM,"TAYLOR, NE",1653.0,5/31/1985,2466,1000,BANK OF TAYLOR,14542.0,13360.0,FAILURE,PA,FDIC
4,18388.0,NM,"FAIRFIELD, NE",1656.0,5/31/1985,6571,1001,FAIRFIELD STATE BANK,6795.0,6034.0,FAILURE,PO,FDIC


In [59]:
data2.columns

Index(['CERT', 'CHCLASS1', 'CITYST', 'COST', 'FAILDATE', 'FIN', 'ID', 'NAME',
       'QBFASSET', 'QBFDEP', 'RESTYPE', 'RESTYPE1', 'SAVR'],
      dtype='object')

Great! The column names match. This makes sense, as it's the same website for the source. (https://banks.data.fdic.gov/explore/failures?aggReport=detail&displayFields=NAME%2CCERT%2CFIN%2CCITYST%2CFAILDATE%2CSAVR%2CRESTYPE%2CCOST%2CRESTYPE1%2CCHCLASS1%2CQBFDEP%2CQBFASSET&endFailYear=2017&sortField=RESTYPE&sortOrder=desc&startFailYear=1934)


In [60]:
data = pd.concat([data, data2], axis=0)

In [61]:
print(len(data))

4677


Don't forget to remove duplicates!

In [62]:
data = data.drop_duplicates()

In [63]:
len(data)

4293

In [64]:
data['RESTYPE'].value_counts()

RESTYPE
FAILURE       3700
ASSISTANCE     593
Name: count, dtype: int64

Courtesy of Google's BARD:

## Here are some ideas for data science projects you can create for your portfolio using data on banks that failed and banks that needed assistance:

1. Develop a predictive model for bank failure. This project would involve using a variety of data mining techniques to identify patterns and risk factors that are associated with bank failure. You could then use these patterns and risk factors to develop a model that can predict the likelihood of failure for other banks.

2. Analyze the factors that contribute to bank failure. This project would involve using statistical techniques to identify the factors that are most strongly associated with bank failure. You could then use this information to develop recommendations for how to prevent bank failures.

3. Develop a risk assessment tool for banks. This project would involve developing a tool that bankers can use to assess the risk of failure for their own banks. The tool could be based on a variety of factors, such as the bank's financial condition, its management practices, and the economic environment.

4. Compare the characteristics of banks that failed to those of banks that needed assistance. This project would involve comparing the characteristics of banks that failed to those of banks that needed assistance. You could then use this information to identify factors that are associated with both failure and assistance.

5. Develop a system for early detection of bank failure. This project would involve developing a system that can identify banks that are at risk of failure before they actually fail. The system could be based on a variety of factors, such as changes in the bank's financial condition, changes in its management practices, or changes in the economic environment.

In [65]:
data.columns

Index(['CERT', 'CHCLASS1', 'CITYST', 'COST', 'FAILDATE', 'FIN', 'ID', 'NAME',
       'QBFASSET', 'QBFDEP', 'RESTYPE', 'RESTYPE1', 'SAVR'],
      dtype='object')

In [75]:
# Convert the FAILDATE column from string to datetime type
data['FAILDATE'] = pd.to_datetime(data['FAILDATE'], errors='coerce')

# Define the target variable
target = pd.to_datetime('2023-11-18') # Replace with the date you want to predict
target_name = 'FAILED' # Replace with the name of the target variable

# Create a new binary variable to indicate whether the bank failed before the target date
data['FAILED'] = (data['FAILDATE'] <= target).astype(int)

# Get dummies for CHCLASS1
encoder = OneHotEncoder()
chclass1_dummies = encoder.fit_transform(data[['CHCLASS1', 'CITYST', 'NAME', 'RESTYPE', 'RESTYPE1', 'SAVR']]).toarray()

# Create a new DataFrame with the original features and the dummy variables
X = pd.DataFrame(data[['CERT', 'COST', 'QBFASSET', 'QBFDEP',]].values)
X = pd.concat([X, pd.DataFrame(chclass1_dummies)], axis=1)

# X = data[['CERT', 'CHCLASS1', 'CITYST', 'COST', 'QBFASSET', 'QBFDEP', 'RESTYPE', 'RESTYPE1', 'SAVR']]
y = data['FAILED']


In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [77]:
X_train

,0,1,2,3,0,1,2,3,4,5,...,6139,6140,6141,6142,6143,6144,6145,6146,6147,6148
3524,20332.0,19062.0,133834.0,125232.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
307,17837.0,2425.0,130411.0,126993.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2565,6499.0,115612.0,1417531.0,1380815.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4270,31284.0,42898.0,277008.0,256379.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1627,32035.0,67673.0,260627.0,263358.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,18243.0,135727.0,285015.0,259056.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3164,NaN,NaN,NaN,1834.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3682,16255.0,4919.0,18169.0,15775.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2413,30645.0,6.0,133547.0,117224.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [78]:
model = LogisticRegression()
model.fit(X_train, y_train)


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [73]:
y_pred = model.predict(X_test)


ValueError: could not convert string to float: 'NM'

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [50]:
# Convert the FAILDATE column from string to datetime type
data['FAILDATE'] = pd.to_datetime(data['FAILDATE'], errors='coerce')

# Define the target variable 
target = pd.to_datetime('2023-11-18') # Replace with the date you want to predict
target_name = 'FAILED' # Replace with the name of the target variable

# Create a new binary variable to indicate whether the bank failed before the target date
data['FAILED'] = (data['FAILDATE'] <= target).astype(int)

# Get dummies for CHCLASS1
encoder = OneHotEncoder()
chclass1_dummies = encoder.fit_transform(data[['CHCLASS1', 'CITYST', 'NAME', 'RESTYPE', 'RESTYPE1', 'SAVR']]).toarray()
# cityst_dummies = encoder.fit_transform()

# Create a new DataFrame with the original features and the dummy variables
X = pd.DataFrame(data[['CERT', 'COST', 'QBFASSET', 'QBFDEP',]].values)
X = pd.concat([X, pd.DataFrame(chclass1_dummies)], axis=1)

# Split the data into training and testing sets
y = data[target_name]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [51]:
# X_train

In [52]:

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred) # Replace with the appropriate error calculation depending on the task
print("Mean Squared Error:", mse)


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# Loading more data
Numbers represent thousands of US Dollars